In [5]:
# Для каждого слова -- кроме самых частотных, служебных, числительных и именованных сущностей
# считаем, в какие даты оно упоминается.
import pickle
import pymorphy2
import pandas as pd
import json
from ner.utils import md5_hashsum, download_untar
from glob import glob
from datetime import date
from ner.network import NER
from ner.corpus import Corpus
from nltk.corpus import stopwords

In [3]:
with open("texts.pkl", "rb") as file:
    data = pickle.load(file)

FileNotFoundError: [Errno 2] No such file or directory: 'texts.pkl'

In [ ]:
data = data.reset_index(drop=True)

In [ ]:
data.head()

In [ ]:
with open("most_frequent.txt", 'r') as file:
    most_freq = [word for word in file.read().split('\n')]

In [ ]:
morph = pymorphy2.MorphAnalyzer()
stopwords = set(stopwords.words('russian'))

def tokenize(text):
    return ([word.strip('.,:;?!()""''') for word in text.split(" ")])

def lemmatize(text):
    return (" ".join([morph.parse(word)[0].normal_form for word in tokenize(text)]))

In [7]:
if md5_hashsum(sorted(glob('model/*'))) != 'fd50a27b96b24cdabdda13795a3baae7':
    # Download and extract model
    download_url = 'http://lnsigo.mipt.ru/export/models/ner/ner_model_total_rus.tar.gz'
    download_path = 'model/'
    download_untar(download_url, download_path)

100%|█████████▉| 44.1M/44.3M [00:24<00:00, 1.80MB/s]

Extracting model/ner_model_total_rus.tar.gz archive into model/


100%|██████████| 44.3M/44.3M [00:40<00:00, 1.80MB/s]

In [6]:
with open('model/params.json') as f:
    network_params = json.load(f)

corpus = Corpus(dicts_filepath='model/dict.txt')

network = NER(corpus, verbouse=False, pretrained_model_filepath='model/ner_model', **network_params)

FileNotFoundError: [Errno 2] No such file or directory: 'model/params.json'

In [ ]:
def is_ok(word):
    if word in stopwords:
        return (False)
    if word in most_freq:
        return (False)
    if 'NUMR' in morph.parse(word)[0].tag:
        return (False)
    return (True)

In [ ]:
reverse_index = {}

In [ ]:
for i in range(len(data["text"])):
    if i % 100 == 0:
        with open("reverse_index.pkl", "wb") as file:
            pickle.dump(reverse_index, file)
    date = data["date"][i]
    text = lemmatize(data["text"][i])
    